In [12]:
import numpy as np

In [13]:
def mwu(data):
    data = np.array(data)
    #data is a T by N matrix with each row being the returns of each expert i at time t
    T = len(data)
    N = len(data[0])
    epsilon = np.sqrt(np.log(N)/T) # is this correct
    #find bounds for loss
    M = max(np.max(data),-1*np.min(data))
    w = np.ones(N)
    #iterations
    for t in range(T):
        #losses
        loss = -1*data[t]/M #positive return is negative loss to increase weight
        #update
        for i in range(N):
            w[i] = w[i]*(1-epsilon*loss[i])
        
    p = w/sum(w)
    return p

array([0.25      , 0.30685323, 0.02258706, 0.4205597 ])